# DESENVOLVENDO SCRIPT PARA LER A IMAGEM E EXTRAIR INFORMACOES

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2

img_beija_flor = r'C:\Users\Plus-TI\Documents\APRENDIZAGEM\OPENCV\Parte1-Filtros\images\beija-flor.jpg'

In [ ]:
img = mpimg.imread(r'C:\Users\Plus-TI\Documents\APRENDIZAGEM\OPENCV\Parte1-Filtros\images\estrada.jpg')
print(f'ESTA IMAGEM É UM OBJETO {type(img)}, com a seguinte dimensoes: {img.shape}')
plt.imshow(img)
plt.show()

In [ ]:
xsize, ysize = img.shape[0], img.shape[1]
print(f'LINHAS:{xsize} COLUNAS:{ysize}')


In [ ]:
color_select = np.copy(img)

In [ ]:
#criterio de selecao de cores
fix = 200
red_threshold = fix
green_threshold = fix
blue_threshold = fix 
rgb_threshold = [red_threshold, green_threshold, blue_threshold]
print(rgb_threshold)

In [ ]:
thresholds = (img[:,:,0] < rgb_threshold[0]) \
            | (img[:,:,1] < rgb_threshold[1]) \
            | (img[:,:,2] < rgb_threshold[2])
color_select[thresholds] = [0, 0, 0]
plt.imshow(color_select)
plt.show()

# APLICANDOO  FILTRO NA IMAGEM

## UTILIZANDO O PIL

In [ ]:
from PIL import Image
from PIL import ImageFilter

In [ ]:
img = Image.open(img_beija_flor)

In [ ]:
img.show()

In [ ]:
img = img.filter(ImageFilter.MedianFilter(11))
img.show()

## UTILIZANDO O OPENCV

In [ ]:
import numpy as np
import cv2

In [ ]:
img = cv2.imread(img_beija_flor)

In [ ]:
cv2.imshow('IMG ORIGINAL', img)
cv2.waitKey(0)

In [ ]:
blurred = np.hstack([cv2.medianBlur(img, 3), cv2.medianBlur(img, 5), cv2.medianBlur(img, 7)])
cv2.imshow('IMG BLUR', blurred)
cv2.waitKey(0)

# MEDIA MODERADA

## APLICANDO NO PIL

In [ ]:
img = Image.open(img_beija_flor)
img = img.filter(ImageFilter.GaussianBlur(3))
img.show()

## APLICANDO NO CV2

In [ ]:
img = cv2.imread(img_beija_flor)
kernels = [(3, 3), (9, 9), (15, 15), (21, 21)]

for (kX, kY) in kernels:
    blurred = cv2.blur(img, (kX, kY))
    cv2.imshow(f'IMG BLUR({kX}{kY})', blurred)
    cv2.waitKey(0)

# OPERACAOES MORFOLOGICAS (EROSAO E DILATACAO)

## EROSAO

In [ ]:
img = cv2.imread(img_beija_flor)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imshow('IMG ORIGINAL CINZA', gray)
cv2.waitKey(0)
for i in range(0, 5):
    erosao = cv2.erode(gray.copy() , None, iterations=i + 1) # parametros: uma copia da img, kernel, interacao
    cv2.imshow(f'EROSAO {i} IMG', erosao)
    cv2.waitKey(0)

## DILATACAO

In [ ]:
img = cv2.imread(img_beija_flor)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

for i in range(0, 5):
    dilatacao = cv2.dilate(gray.copy(), None, iterations=i+1)# parametros: uma copia da img, kernel, interacao
    cv2.imshow(f'DILATACAO {i} IMG', dilatacao)
    cv2.waitKey(0)

# FILTRO PERSONALIZADOS

In [ ]:
img = Image.open(img_beija_flor)
kernal = ImageFilter.Kernel((3, 3), [1, 0, -1, 5, 0, -5, 1, 0, 1])
print(kernal)
img = img.filter(kernal)
img.show()

# THRESHOLD
### O CONCEITO DE THRESHOLD É SEPARAR A IMAGEM DE FUNDO COM A IMAGEM DE INTERESSE. PARA ISSO PODE SE BINARIZAR A IMAGEM E ATRIBUIR DUAS CORES PARA DESTACAR O QUE NÃO É RELEVANTE DO QUE É. NORMALMENTE ATRAVÉS DE UMA OPERAÇÃO DE HISTOGRAMA O ALGORITIMO PEGA O VALOR MEDIO DO GRAFICO PARA CRIAR O THESHOLD

In [ ]:
img = cv2.imread(img_beija_flor)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#ATRAVES DO FILTRO GAUSIANO, PODEMOS TER UMA PREPARACAO PARA EXTRAIR O QUE DESEJAMOS. TAMBÉM É UTIL PARA SUAVISAR OS EXCESSO DE PIXELIS NA IMAGEM. SEM ISSO A IMAGEM CORRE O RISCO DE FICAR MUITO CLARA.
blurred = cv2.GaussianBlur(img, (5, 5), 0)
cv2.imshow('IMG COM FILTRO GAUSSIANO', blurred)

(T, thresh) = cv2.threshold(blurred, 155, 255, cv2.THRESH_BINARY)# PARAMETROS: imagem, limite (limiar), se o valor do pixel for maior que o limiar convertemos para BRANCO, caso contrario para preto. O proximo parametro e o valor que vai ser usado para valores acima do campo anterior. O ultimo campo vai ser o algoritimo que vai ser usado. Veja que abaixo ele vai fazer o mesmo só que invertendo o resultado.
cv2.imshow('IMAGEM BINARIZADA COM THRESHOLD', thresh)

(T, threshoInv) = cv2.threshold(blurred, 155, 255, cv2.THRESH_BINARY_INV)
cv2.imshow('IMAGEM BINARIZADA COM THRESHOLD INVERTIDA', threshoInv)

cv2.imshow('CASAS DECIMAIS', cv2.bitwise_and(img, img, mask=threshoInv))
cv2.waitKey(0)



### NA IMAGEM DA CELULA ACIMA VIMOS UM DEFEITO MUITO SERIO NA QUESTÃO DA PERCA DOS DETALHES, ISSO ACONTECE POR CONTA DO VALOR DE THRESHOLD(155) QUE NAO FOI O MELHOR VALOR, ISSO OCASIONA DE FICAR TENTANDO ENCONTRAR UM VALOR QUE SEJA MAIS ADEQUADO. PARA RESOLVER ISSO O CV2 TEM UMA FORMA DE ENCONTRAR DE FORMA INTELIGENTE.

In [ ]:
img = cv2.imread(img_beija_flor)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

blurred = cv2.GaussianBlur(img, (5, 5), 0)
cv2.imshow('IMAGEM ORIGINAL', blurred)

thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 4)

cv2.imshow('IMAGEM THRESHOLD', thresh)

thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 15, 3)

cv2.imshow('GAUSSIANA THRESHOLD', thresh)
cv2.waitKey(0)

# DETECCAO DE BORDAS(SOBEL)

In [ ]:
img = cv2.imread(r'C:\Users\Plus-TI\Documents\APRENDIZAGEM\OPENCV\mini projeto\imagens_teste\imagem2.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
cv2.imshow('IMAGEM ORIGINAL', gray)
cv2.waitKey(0)


#COMPUTA OS GRADIENTES
gX = cv2.Sobel(gray, ddepth=cv2.CV_64F, dx=1, dy=0)
gY = cv2.Sobel(gray, ddepth=cv2.CV_64F, dx=0, dy=1)
print(gX, gY, type(gX), type(gY))
# AS IMAGENS gX e gY  SAO NUMEROS FLUTUANTES, PRECISAMOS CONVERTER NOVAMENTE

gX = cv2.convertScaleAbs(gX)
gY = cv2.convertScaleAbs(gY)

#COBINA AS REPRETACOES Sobel x e y em uma unica imagems
sobelCombinado = cv2.addWeighted(gX, 0.5, gY, 0.5, 0)

cv2.imshow('SOBEL X', gX)
cv2.imshow('SOBEL Y', gY)
cv2.imshow('SOBEL COMBINADO', sobelCombinado)
cv2.waitKey(0)


# CALCULAR O GRADIENTE DE X E Y PARA DETECTAR DENTRO DA BORDA(MAGNITUDE E ORIENTACAO)

In [ ]:
img = cv2.imread(img_beija_flor)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imshow('ORIGINAL', img)
cv2.waitKey(0)

#COMPUTADONDO O GRADIENTE DE X E Y
gX = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
gY = cv2.Sobel(gray, cv2.CV_64F, 0, 1)

mag = np.sqrt((gX ** 2) + (gY ** 2))
orientacao = np.arctan2(gY, gX) * (180 / np.pi) % 180
print(f'mag: {mag} orientacao: {orientacao}')

#ENCONTRAR TODOS OS PIXELS QUE ESTAO DENTRO DOS LMITES DOS ANGULOS
idx = np.where(orientacao >= 175.0, orientacao, -1)
idx = np.where(orientacao <= 180.0, idx, -1)

mascara = np.zeros(gray.shape, dtype='uint8')
mascara[idx > -1] = 255

cv2.imshow('BORDAS DA IMAGEM', mascara)
cv2.waitKey(0)


# CANNY ADGE DECTETOR (DETECTOR DE BORDAS)
### ACRESCENTA ALGUMAS ETAPAS PARA ENCONTRAR AS BORDAS:
### 1 SUAVIZACAO DAS BORDAS
### 2 ENCONTRAMOS O GRADIENTES
### 3 VERIFICAR SE O GRADIENTE CALCULADO TEM PONTOS VISINHOS 
### 4 THRESHOLD (LIMITE LIMIAR)

In [ ]:
img = cv2.imread(r'C:\Users\Plus-TI\Documents\APRENDIZAGEM\OPENCV\Parte1-Filtros\images\estrada.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# REALIZA A DESFOCAGEM OU SUAVIZACAO DAS BORDAS
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

cv2.imshow('ORIGINAL', blurred)

# CALCULA UM LIMITE 'WIDE', 'MID-RANGE' E 'TIGHT' PARA AS BORDAS
wide = cv2.Canny(blurred, 10, 200)
mid = cv2.Canny(blurred, 30, 150)
tight = cv2.Canny(blurred, 240, 250)

cv2.imshow('IMAGEM WIDE', wide)
cv2.imshow('IMAGEM MID', mid)
cv2.imshow('IMAGEM TIGHT', tight)
cv2.waitKey(0)



In [ ]:
### DEFININDO O VALOR AUTOMATICO PARA ELE ENCONTRAR AS BORDAS...

In [29]:
def autoCanny(img, sigma=.33):
    'APLICA A MEDIANA ATRAVES DA INTENSIDADES DE PIXEL DE UM UNICO CANAL'
    v = np.median(img)
    'APLICANDO A DETECCAO AUTOMATICA DE BORDAS CANNY USANDO A MEDIANA'
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(img, lower, upper)
    return edged

img = cv2.imread(r'C:\Users\Plus-TI\Documents\APRENDIZAGEM\OPENCV\Parte1-Filtros\images\casa.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (3, 3), 0)

#APLICANDO OS CANNY ANTERIORES E O AUTOCANNY PARA PERCEBER A DIFERENCA
wide = cv2.Canny(blurred, 10, 200)
tight = cv2.Canny(blurred, 255, 250)
auto = autoCanny(blurred)

cv2.imshow('IMG ORIGINAL', blurred)
cv2.imshow('IMG WIDE', wide)
cv2.imshow('IMG TIGHT', tight)
cv2.imshow('IMG AUTO', auto)
cv2.waitKey(0)

-1

# DESENHAR SOBRE A BORDAS ENCONTRADAS

In [54]:
def autoCanny(img, sigma=.33):
    'APLICA A MEDIANA ATRAVES DA INTENSIDADES DE PIXEL DE UM UNICO CANAL'
    v = np.median(img)
    'APLICANDO A DETECCAO AUTOMATICA DE BORDAS CANNY USANDO A MEDIANA'
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(img, lower, upper)
    return edged

img = cv2.imread(r'C:\Users\Plus-TI\Documents\APRENDIZAGEM\OPENCV\mini projeto\imagens_teste\imagem2.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

'DESFOCAGEM'
blurred = cv2.GaussianBlur(gray, (11, 11), 0)

'CANNY'
edged = autoCanny(blurred)

cv2.imshow('IMG ORIGINAL', img)
cv2.waitKey(0)
#ENCONTRA TODOS OS CONTORNOS NA IMAGEM E DESENHA TODOS OS CONTORNOS NA IMAGEM
(_, cnts) = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE) # PARAMETROS: UMA COPIA DA IMAGEM, FLAG QUE GARANTE TODOS OS CONTORNOS, RETORNA OS DETALHES MAIS SIMPLES DOS PIXELS.

#FUNCAO findCountrs É DESTRUTIVA, OU SEJA, ELA ALTERA A IMAGEM. POR ISSO FAZEMOS UMA COPIA DA IMAGEM

clone = img.copy()

#DESENHAR OS CONTORNOS
cv2.drawContours(clone, _, -1, (0, 255, 0), 2) # PARAMETROS: 1- IMAGEM, 2 - LISTA DE CONTORNOS, 4 - COR DESEJADA
print(f'ENCONTRADO {len(_)} CONTORNOS')

cv2.imshow('IMAGEM COM CONTORNOS', clone)
cv2.waitKey(0)

ENCONTRADO 327 CONTORNOS


True

# CONTAGEM DE OBJETOS NA IMAGEM

In [90]:
img = cv2.imread(r'C:\Users\Plus-TI\Documents\APRENDIZAGEM\OPENCV\Parte2-EdgeDetection\images\shapes.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#DESSA VEZ ESTOU PEGANDO A BORDA EXTERNA
qtde, _ = cv2.findContours(gray.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
clone = img.copy()

for (i, c) in enumerate(qtde):
    # CALCULA AREA E PERIMETRO DO CONTORNO
    area = cv2.contourArea(c)
    perimetro = cv2.arcLength(c, True)
    print(f'CONTORNO {i+1} AREA {area} PERIMETRO {perimetro}')

    #DESENHANDO O CONTORNO NA IMAGEM
    cv2.drawContours(clone, [c], -1, (0, 255, 0), 2)

    #CALCULANDO O CENTRO DO CONTORNO E DESENHA O NUMERO DO CONTORNO
    m = cv2.moments(c)
    cX = int(m['m10'] / m['m00'])
    cY = int(m['m01'] / m['m00'])
    cv2.putText(clone, f'{i+1}', (cX - 20, cY), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (255, 255, 255), 4)

cv2.imshow('IMAGEM CONTADA', clone)
cv2.waitKey(0)


CONTORNO 1 AREA 200646.0 PERIMETRO 1906.0


-1

In [94]:
# VERIFICANDO SE O OBJETO É RETANGULO OU 
img = cv2.imread(r'C:\Users\Plus-TI\Documents\APRENDIZAGEM\OPENCV\Parte2-EdgeDetection\images\shapes.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

(qtde, _) = cv2.findContours(gray.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# SEGUIR COM A CONTAGEM E DESENHAR O CONTORNO
for cc, c in enumerate(qtde):
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.01 * peri, True)

    #SE O CONTORNO TIVER 4 VERTICES ENTÃO ESTAMOS EXAMINANDO UM RETANGULO
    if(len(approx) == 4):
        cv2.drawContours(img, [c], -1, (0, 255, 0), 2)
        (x, y, w, h) = cv2.boundingRect(approx)
        cv2.putText(img, f'RETANGULO({cc})', (x, y - 10),       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    else:
        cv2.drawContours(img, [c], -1, (255, 0, 0), 2)
        (x, y, w, h) = cv2.boundingRect(approx)
        cv2.putText(img, f'PROVAVEL CIRCULO:({cc})', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
cv2.imshow('IMAGEM COM A CONTAGEM E VERICACAO', img)
cv2.waitKey(0)
    

-1

### CAPTURANDO COM WEBCAM

In [103]:
captura = cv2.VideoCapture(0)

while True:
    rect, frame = captura.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    wide = cv2.Canny(blurred, 10, 200)

    cv2.imshow('IMAGEM', wide)
    k = cv2.waitKey(30) & 0xff
    if(k == 27): break
captura.release()
cv2.destroyAllWindows()